**Note: GitHub profiles with many repositories may result in error due to GitHub API rate limits**

GitHub profiles of the following users have been compared

https://github.com/ps-2810 - Back end developer

https://github.com/kkistheorz - Front end developer

#Define Parameters

In [ ]:
task = 'Create a Task Manager Web App that allows users to efficiently manage their daily tasks'
num_recommend = 2 #number of users to be recommended
top_k = 1 #Average of top_k matching sentences

#Get the usernames from github profiles and make a dataframe (you can also use a file with a list of github usernames)

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
developers = ['https://github.com/ps-2810', 'https://github.com/kkistheorz']

In [ ]:
usernames = []
for devs in developers:
  usernames.append(devs.split('/')[3])

Upload file below (if you already have a list of usernames)

In [ ]:
df = pd.DataFrame()
df['dev'] = usernames

#Get repository information from GitHub

##Importing libraries

In [ ]:
!pip install PyGithub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: pyjwt
    Found existing installation: PyJWT 2.3.0
    Uninstalling PyJWT-2.3.0:
      Successfully uninstalled PyJWT-2.3.0


In [ ]:
!pip install gensim==3.7.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 18.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gensim: filename=gensim-3.7.0-cp310-cp310-linux_x86_64.whl size=24585051 sha256=d7918355bc020d95fbd52521a22ec69c1b21543183a043b3c901c0a108d4c44b
  Stored in directory: /root/.cache/pip/wheels/dc/47/f9/022929f4b4de4a3be6f7f59ecef50db175700f594787f7caed
Successfully built gensim
  Attempting uninstall: gensim
    Found existing installation: gensim 4.3.2
    Uninstalling gensim-4.3.2:
      Successfully uninstalled gensim-4.3.2


There is a problem with gensim package.
Two ways to fix it:
https://drive.google.com/drive/folders/1h3hg8jdVhL3pPkF3Zww700vXqzpKcQ-g?usp=sharing
1. Use the gensim package used in the above zip file

OR
2. In the dictionary.py file in the gensim 3.7.0 package:
replace
**from collections import Mapping, defaultdict**

to

**from collections.abc import Mapping**

**from collections import defaultdict**

In [ ]:
import gensim
from gensim.models.doc2vec import Doc2Vec

In [ ]:
print("gensim", gensim.__version__)

gensim 3.7.0


In [ ]:
import pandas as pd
import requests
from github import Github
from github import RateLimitExceededException
from github import UnknownObjectException
from datetime import datetime, timedelta
import pickle
import time
import itertools
import os
import calendar
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import requests
import base64

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


##Downloading the pretrained dev2vec model

In [ ]:
!gdown 1pfU1uwe5qXw1H6nFYzeobFwBZUBfy_f0
!gdown 1Ie-9pc5qCyC8FHqe6S620C5s_gaWmyvT
!gdown 13OW3YenkWXRDjEhyUnq0FRhWASlaCbAP
!gdown 1nBJZzuIgy7ujeEoHnvDvbVadLCcEOxn_
!gdown 1RCVSnc9SkQP0Qh5mvjKKNJdWghddgGPT
!gdown 1R5B1urqgYPhu-89FP5e0I0VEqH9L9REr
!gdown 19nmPPoIUxxycGqXETPt4jga9yS_4ozVS
!gdown 16x9JlHxY8yH1cM8f5M5gwKwZzH5OOCKN

Downloading...
From: https://drive.google.com/uc?id=1pfU1uwe5qXw1H6nFYzeobFwBZUBfy_f0
To: /content/dev2vec_issues.wv.vectors.npy
100% 4.26M/4.26M [00:00<00:00, 67.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Ie-9pc5qCyC8FHqe6S620C5s_gaWmyvT
To: /content/dev2vec_issues
100% 563k/563k [00:00<00:00, 69.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=13OW3YenkWXRDjEhyUnq0FRhWASlaCbAP
To: /content/dev2vec_repos.trainables.syn1neg.npy
100% 19.7M/19.7M [00:00<00:00, 186MB/s]
Downloading...
From: https://drive.google.com/uc?id=1nBJZzuIgy7ujeEoHnvDvbVadLCcEOxn_
To: /content/dev2vec_repos
100% 1.58M/1.58M [00:00<00:00, 170MB/s]
Downloading...
From: https://drive.google.com/uc?id=1RCVSnc9SkQP0Qh5mvjKKNJdWghddgGPT
To: /content/dev2vec_issues.trainables.syn1neg.npy
100% 4.26M/4.26M [00:00<00:00, 250MB/s]
Downloading...
From: https://drive.google.com/uc?id=1R5B1urqgYPhu-89FP5e0I0VEqH9L9REr
To: /content/dev2vec_repos.docvecs.vectors_docs.npy
100% 1.19M/1.19M [00:00<00:00, 14

##Get the repository information

In [ ]:
def get_github_repositories(username):
    url = f'https://api.github.com/users/{username}/repos'
    response = requests.get(url)

    if response.status_code == 200:
        repositories = response.json()
        return repositories
    else:
        print(f"Error: Unable to fetch repositories. Status code: {response.status_code}")
        return None

def get_repository_info(username, repo_name):
    url = f'https://api.github.com/repos/{username}/{repo_name}'
    response = requests.get(url)

    if response.status_code == 200:
        repository_info = response.json()
        return repository_info
    else:
        print(f"Error: Unable to fetch repository information. Status code: {response.status_code}")
        return None

def get_readme_content(username, repo_name):
    url = f'https://api.github.com/repos/{username}/{repo_name}/readme'
    response = requests.get(url)

    if response.status_code == 200:
        readme_info = response.json()
        readme_content_base64 = readme_info.get('content', '')
        readme_content = base64.b64decode(readme_content_base64).decode('utf-8')
        return readme_content
    else:
        print(f"Error: Unable to fetch README content. Status code: {response.status_code}")
        return None

In [ ]:
def repos(github_username):

  repositories = get_github_repositories(github_username)

  if repositories:
      repo_info_array = []

      for repo in repositories:
          repo_name = repo['name']
          repo_info = get_repository_info(github_username, repo_name)

          if repo_info:
              description = repo_info.get('description', 'N/A')

              readme_content = get_readme_content(github_username, repo_name)
              if readme_content:
                  full_info = f"{description}. {readme_content}"
              else:
                  full_info = f"{description}. Unable to fetch README content."

              repo_info_array.append(full_info)
          else:
              print(f"Unable to fetch information for repository: {repo_name}")

      # Print or use repo_info_array as needed
      print("Repository Information Array:")
      print(repo_info_array)
      return repo_info_array
  else:
      return ['NA']

##Preprocess repository information text

In [ ]:
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove punctuation and convert to lowercase
    tokens = [word.lower() for word in tokens if word.isalpha()]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return tokens

In [ ]:
dev_info = []
for i in range(len(df['dev'])):
  repo_concat = (' '.join(repos(df['dev'][i]))).split('.')
  dev_info.append(repo_concat)

Repository Information Array:
['This is a backend project. # Project Description:\nThe project will have the following endpoints:\n\nPOST /register: Register a new user with a username and password.\nPOST /login: Authenticate a user with a registered username and password.\n\n# Project Setup:\n1. Create a new directory for your project and navigate to it in the terminal.\n```\nmkdir simple-auth-api\ncd simple-auth-api\n```\n2. Initialize a new Node.js project.\n```\nnpm init -y\n```\n3. Install the necessary dependencies (Express and bcrypt for password hashing).\n```\nnpm install express bcrypt\n4. Running the Project: Save the changes to index.js and Run the server\n```\nnode index.js\n```\n', 'None. # potterhead\nExpress backend that creates a simple REST API for managing tasks. This project includes endpoints for retrieving a list of tasks, getting a specific task by ID, creating a new task, updating an existing task, and deleting a task. The tasks are stored in-memory for simplici

#Implementing Dev2Vec

##Vectorizing the developers and the task

In [ ]:
model = Doc2Vec.load("dev2vec_repos")

In [ ]:
dev_embed = [[None] * len(inner_list) for inner_list in dev_info]

In [ ]:
for i in range(len(dev_info)):
  dummy = np.empty_like(dev_info[i])
  for j in range(len(dev_info[i])):
    dummy = model.infer_vector(preprocess_text(dev_info[i][j]))
    dev_embed[i][j] = dummy

In [ ]:
task_embed = model.infer_vector(preprocess_text(task))

##Install and import the required libraries

In [ ]:
!pip install faiss-gpu
!pip install tensorflow_hub
!pip install hnswlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp310-cp310-linux_x86_64.whl size=2287618 sha256=f46249999db8bc0679c03a8ea736781fab06db35963e8aa734fadb919d4271d2
  Stored in directory: /root/.cache/pip/wheels/af/a9/3e/3e5d59ee41664eb31a4e6de67d1846f86d16d93c45f277c4e7
Successfully built hnswlib


In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
import hnswlib

##Get k nearest neighbours using hnswlib

In [ ]:
labels = np.empty_like(dev_info)
distances = np.empty_like(dev_info)
for i in range(len(df['dev'])):
  dev_tensor = tf.convert_to_tensor(dev_embed[i], dtype=tf.float32)
  keyword_embeds = tf.convert_to_tensor(task_embed, dtype=tf.float32)
  num_elements = len(dev_embed[i])
  ids = np.arange(num_elements)
  dim = dev_tensor.shape[1]
  p = hnswlib.Index(space = 'l2', dim = dim)
  p.init_index(max_elements = num_elements, ef_construction = 200, M = 16)
  p.add_items(dev_embed[i], ids)
  p.set_ef(50) # ef should always be > k
  labels[i], distances[i] = p.knn_query(keyword_embeds, k = len(dev_embed[i]))

<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


##Resulting priority of developers

In [ ]:
distance_avg = np.empty(len(df['dev']))
for i in range(len(df['dev'])):
  if(len(distances[i][0])<top_k):
    distance_avg[i] = (np.sum(distances[i][0][:len(distances[i][0])]))/len(distances[i][0])
  else:
    distance_avg[i] = (np.sum(distances[i][0][:top_k]))/top_k

In [ ]:
def sort_with_indices(arr):
    indexed_arr = list(enumerate(arr))
    sorted_arr = sorted(indexed_arr, key=lambda x: x[1])
    sorted_indices = [index for index, _ in sorted_arr]
    return sorted_indices

priority_list = sort_with_indices(distance_avg)

print('Priority Order is as follows:')
if(num_recommend<=len(df['dev'])):
  for i in range(num_recommend):
    print(df['dev'][priority_list[i]])
else:
  print('Number asked for recommendation more than existing students')

Priority Order is as follows:
kkistheorz
ps-2810
